In [5]:
# Suppress dataframe concat warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import yaml
import os
import numpy as np

class DataFolder():
    def __init__(self, 
                 folder_path, 
                 labels_path="./標籤集合.xlsx", 
                 prices_path="./圖片的價格.xlsx"):
        '''
        folder_path: yolov7pytorch folder
        '''
        self.folder_path = folder_path
        self.labels = pd.DataFrame()
        self.prices = pd.DataFrame()
        self.side_dishes = []
        self.rices = []
        self.containers = []
        self.main_dishes = []
        self.class_id_to_name = {}
        
        self.train_data = pd.DataFrame()
        self.valid_data = pd.DataFrame()
        self.test_data = pd.DataFrame()

        self.load_labels(labels_path)
        self.load_prices(prices_path)
        self.load_class_id_to_name(folder_path + "/data.yaml")
        self.load_train(folder_path + "/train")
        self.load_valid(folder_path + "/valid")
        self.load_test(folder_path + "/test")
        
        self._check_labels()

    def _check_labels(self):
        # Check if the last column is diff
        # If not, than theres something missing with the labels xlsx
        if self.train_data.columns[-1] != "diff":
            print("The last column of train data should be diff.")
            print("But it is...", self.train_data.columns[-5:])
        if self.valid_data.columns[-1] != "diff":
            print("The last column of valid data should be diff.")
            print("But it is...", self.valid_data.columns[-5:])
        if self.test_data.columns[-1] != "diff":
            print("The last column of test data should be diff.")
            print("But it is...", self.test_data.columns[-5:])

    def load_labels(self, path):
        if path.endswith('.xlsx'):
            self.labels = pd.read_excel(path)
        else:
            print('File format not supported')
            return
        
        # Get unique labels
        self.side_dishes = self.labels["副菜"].dropna().unique().tolist()
        self.rices = self.labels["飯"].dropna().unique().tolist()
        self.containers = self.labels["容器"].dropna().unique().tolist()
        self.main_dishes = self.labels["主菜"].dropna().unique().tolist()
        
        # Make a dic store the class name and its price
        dic = {}
        for item in self.side_dishes + self.rices + self.main_dishes:
            dic[item.split('(')[0]] = int(item.split(') ')[1].split('(')[0])
        
        # Leave english labels
        self.side_dishes = [item.split('(')[0] for item in self.side_dishes]
        self.rices = [item.split('(')[0] for item in self.rices]
        self.containers = [item.split('(')[0] for item in self.containers]
        self.main_dishes = [item.split('(')[0] for item in self.main_dishes]
        
    def load_prices(self, path):
        if path.endswith('.xlsx'):
            self.prices = pd.read_excel(path)
        else:
            print('File format not supported')

    def load_class_id_to_name(self, path):
        self._load_yaml(path)

    def load_train(self, path):
        self.train_data = self._load_data(path)

    def load_valid(self, path):
        self.valid_data = self._load_data(path)

    def load_test(self, path):
        self.test_data = self._load_data(path)

    def _load_data(self, path):
        '''
        dataframe columns:
            * side_dishes: sum of the area ratio of each side dish
            * rices: one hot encoding 3 types of rice
            * containers: one hot encoding 2 types of container
            * main_dishes: one hot encoding >50 types of main dishes
            * side_dishes_n: number of side dishes
            * price: 
            * fair price: 
            * diff: price - fair price
        '''
        col = self.side_dishes + self.rices + self.containers + self.main_dishes
        data = pd.DataFrame(columns=col)
        containerless_n = 0
        priceless_n = 0
        
        # For each image label file
        for filename in os.listdir(path + "/labels"):
            if not filename.endswith(".txt"):
                print(f"{filename} is not a label file.")
                continue

            df = pd.read_csv(path + "/labels/" + filename, 
                             sep=" ", 
                             header=None,
                             names=["class_id", "x_center", "y_center", "width", "height"])
            df["class_name"] = df["class_id"].apply(lambda x: self.class_id_to_name[x])
            df["area"] = df.apply(lambda x: self._cal_area(x["width"], x["height"]), axis=1)
            
            # If has container, calculate the ratio
            if df[df["class_name"].isin(self.containers)]["area"].sum() != 0:
                df["area_ratio"] = df["area"] / df[df["class_name"].isin(self.containers)]["area"].sum()
            # Else calculate the ratio of all
            else: 
                df["area_ratio"] = df["area"] / df["area"].sum()
                containerless_n += 1
            
            # In case bad name
            try:
                # Get the price
                price = self.prices[self.prices["filename"] == filename.split("_jpg")[0]]["price"].values[0]
                # Get the fair price
                fair_price = self.prices[self.prices["filename"] == filename.split("_jpg")[0]]["fair price"].values[0]
            except:
                print(filename)
                price = 69
            
            # If the price is not available, replace it with the fair price
            if pd.isna(price):
                price = fair_price
                priceless_n += 1

            # Make a new row
            dic = {}
            for index, row in df.iterrows():
                # Sum of the area ratio of each class
                dic[row["class_name"]] = dic.get(row["class_name"], 0) + row["area_ratio"]
            dic["side_dishes_n"] = len(df[df["class_name"].isin(self.side_dishes)])
            dic["price"] = price
            dic["fair price"] = fair_price
            dic["diff"] = price - fair_price

            # Concat the new row to the data, index is the filename, fill na with 0
            data = pd.concat([data, pd.DataFrame(dic, index=[filename.split("_jpg")[0]])]).fillna(0)

            if "2023_11_1_24" in filename:
                print(dic)
        
        print(f"There are {len(data)} image label files in the dataset {path}.")
        print(f"There are {containerless_n} no container in the dataset.")
        print(f"There are {priceless_n} no price in the dataset.")
        
        return data
    
    def _load_yaml(self, path):
        with open(path, 'r') as f:
            try:
                data = yaml.safe_load(f)
                print(f"There are {data['nc']} classes in the dataset from yaml.")
                self.class_id_to_name = data["names"]
            except yaml.YAMLError as exc:
                print(exc)

    def _cal_area(self, width, height):
        return width * height

In [6]:
datafolder = DataFolder("yolov7pytorch")

There are 52 classes in the dataset from yaml.
There are 1788 image label files in the dataset yolov7pytorch/train.
There are 6 no container in the dataset.
There are 66 no price in the dataset.
There are 283 image label files in the dataset yolov7pytorch/valid.
There are 4 no container in the dataset.
There are 11 no price in the dataset.
{'box': 1.0, 'fried chicken leg steak': 0.27430499093908883, 'white rice': 0.29684342545535036, 'side dish': 0.452181249966645, 'side_dishes_n': 3, 'price': 75.0, 'fair price': 75, 'diff': 0.0}
There are 137 image label files in the dataset yolov7pytorch/test.
There are 1 no container in the dataset.
There are 4 no price in the dataset.


In [7]:
# Apply linear regression
from sklearn.linear_model import LinearRegression

def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

model = LinearRegression()

# Train with diff
model.fit(datafolder.train_data.drop(["price", "fair price", "diff"], axis=1), datafolder.train_data["diff"])
for col, coef in list(zip(datafolder.train_data.drop(["price", "fair price", "diff"], axis=1).columns, model.coef_)):
    print(col, coef)
print(model.intercept_)

print("Train with diff--------------")

# MAPE with valid_data
y_hat = model.predict(datafolder.valid_data.drop(["price", "fair price", "diff"], axis=1))
# Add back fair price
y_hat += datafolder.valid_data["fair price"]
#print(datafolder.valid_data["price"])
print("MAPE with valid data:", MAPE(datafolder.valid_data["price"], y_hat))

# Output to csv: [filename, price, fair price, diff, y_hat, y_hat + fair price, round_to_the_nearest_integer]
output = pd.DataFrame(index=datafolder.valid_data.index)
output["filename"] = datafolder.valid_data.index
output["price"] = datafolder.valid_data["price"]
output["fair price"] = datafolder.valid_data["fair price"]
output["diff"] = datafolder.valid_data["diff"]
output["y_hat(fair price+diff)"] = y_hat
output["rounded_5"] = output["y_hat(fair price+diff)"].apply(lambda x: round(x/5)*5)
output.to_csv("./output/1130_valid_data.csv", index=False)

# MAPE with test_data
y_hat = model.predict(datafolder.test_data.drop(["price", "fair price", "diff"], axis=1))
# Add back fair price
y_hat += datafolder.test_data["fair price"]
print("MAPE with test data:", MAPE(datafolder.test_data["price"], y_hat))

# Output to csv: [filename, price, fair price, diff, y_hat, y_hat + fair price, round_to_the_nearest_integer]
output = pd.DataFrame(index=datafolder.test_data.index)
output["filename"] = datafolder.test_data.index
output["price"] = datafolder.test_data["price"]
output["fair price"] = datafolder.test_data["fair price"]
output["diff"] = datafolder.test_data["diff"]
output["y_hat(fair price+diff)"] = y_hat
output["rounded_5"] = output["y_hat(fair price+diff)"].apply(lambda x: round(x/5)*5)
output.to_csv("./output/1130_test_data.csv", index=False)


side dish 12.850802720496656
white rice -19.435402045566786
purple rice -19.65196773307013
brown rice -23.34130858902677
plate -1.578149807896895
box -2.270513047925135
grilled mackerel 11.856868963083883
salmon -0.5523185306332081
braised fish fillet 6.047623665904862
steamed fish fillet -0.35541200271572193
sweet and spicy fried pork chops 88656632064773.8
cuttlefish steak -18.64213437158028
fried cod steak -2.8007475345917427
sweet and spicy chicken steak 26.537195911962545
chicken steak -1.7723385710065287
fried pork chops -4.137799711501959
braised pork ribs -34.13215336165589
grilled chicken leg steak -0.9064400630184629
honey grilled chicken leg 3.27077862760469
grilled chicken steak 16.978593478229655
big fried chicken leg -3.9464513686475904
small fried chicken leg 1.098382781373985
big grilled chicken leg 3.379624271053016
small grilled chicken leg 1.5105403257990382
stir-fried minced pork 31.521435827803337
pork with scrambled eggs 14.74249655759239
sauteed pork 11.325166357